# Библиотека requests

In [1]:
import requests
import pandas as pd

In [2]:
# метод get
res = requests.get('https://yandex.ru')
res
# res.status_code

<Response [200]>

In [3]:
# браузер отрисовал бы страницу на основе данного текста
res.text

In [4]:
# посмотрим куки
res.cookies

In [ ]:
# получаем плохой статус
bad_request = requests.get('https://yandex.ru/secret')
bad_request

In [ ]:
bad_request.text

In [10]:
# попроуем сделать post запрос
post_req = requests.post('https://yandex.ru')
post_req

<Response [403]>

In [11]:
# cформируем поисковый запрос, обратите внимание на его формат
URL = 'https://yandex.ru/search/?text=python&lr=54'

In [12]:
req = requests.get(URL)
req

<Response [200]>

In [ ]:
req.text

In [ ]:
# в request можно передать параметры запроса и заголовки (headers) в виде словарей. 
# сегодня не будем рассматривать примеры с необхожимостью передачи заголовка, 
# но в практике вам это точно понадобится
URL = 'https://yandex.ru/search/'
params = {
    'text': 'python',
    'lr': 54
}
headers = {}
req = requests.get(URL, params)
req = requests.post(URL, params, headers)
req.text

In [2]:
# очень часто сайты могут ограничивать частые запросы к себе, 
# поэтому нужно задерживать исполнение
import time
time.sleep(0.2)

# Beautiful Soup

In [3]:
# как разбирать всю эту разметку? Поможет BeautifulSoup.
from bs4 import BeautifulSoup

## Практика 1. Напишем скрипт, который будет отбирать посты из нужных хабов на habr.com

In [ ]:
# определяем список хабов, которые нам интересны
DESIRED_HUBS = ['python', 'bigdata']

In [ ]:
# получаем страницу с самыми свежими постами
req = requests.get('https://habr.com/ru/all/')
soup = BeautifulSoup(req.text, 'html.parser')

In [ ]:
# извлекаем посты
posts = soup.find_all('article', class_='post')
posts

In [ ]:
for post in posts:
    post_id = post.parent.attrs.get('id')
   # если идентификатор не найден, это что-то странное, пропускаем
    if not post_id:
        continue
    post_id = int(post_id.split('_')[-1])
    print('post', post_id)
    hubs = post.find_all('a', class_='hub-link')

In [ ]:
# добавляем извлечение хабов из постов, чтобы отбирать только нужные
for post in posts:
    post_id = post.parent.attrs.get('id')
   # если идентификатор не найден, это что-то странное, пропускаем
    if not post_id:
        continue
    post_id = int(post_id.split('_')[-1])
    hubs = post.find_all('a', class_='hub-link')
    for hub in hubs:
            hub_lower = hub.text.lower()
           # ищем вхождение хотя бы одного желаемого хаба
            if any([hub_lower in desired for desired in DESIRED_HUBS]):
               # пост нам интересен - делаем с ним все что захотим:
               # можно отправит в телеграм уведомление, можно на почту и т.п.
                title_element = post.find('a', class_='post__title_link')
                print(title_element.text, title_element.attrs.get('href'))
               # так как пост уже нам подошел - дальше нет смысла проверять хабы
                break

In [ ]:
ДОМАШНЕЕ ЗАДАНИЕ 1

In [4]:
import datetime 

In [9]:
URL = 'https://habr.com/ru/all/'
KEYWORDS = ['python', 'парсинг']

rez_py = requests.get(URL)
soup_py = BeautifulSoup(rez_py.text, 'html.parser')

articles = soup_py.find_all('article', class_ = "post post_preview")

article_list = []
KEYWORDS = ['python', 'парсинг']
for el in articles:
    if KEYWORDS[0] in str(el) or KEYWORDS[1] in str(el):
        article_list.append(el)


article_date = list(map(lambda x: x.find('span', class_ = "post__time"), article_list))
#article_title = list(map(lambda x: x.find('h2', class_ = "post__title"), article_list))
article_title = list(map(lambda x: x.find('h2', class_ = "post__title").get_text(), article_list))
article_link = list(map(lambda x: x.find('a').get('href'), article_list))

df = pd.DataFrame({'Дата': article_date, 'Название':article_title, 'Ссылка':article_link })

df


,Дата,Название,Ссылка
0,[сегодня в 17:03],"\nСтабилизация видео с движущейся камеры, или ...",https://habr.com/ru/users/rd_oxagile/


In [ ]:
   ДОМАШНЕЕ ЗАДАНИЕ 2

In [ ]:

Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса Avast Hack Ckeck.
Список email-ов задаем переменной в начале кода:
EMAIL = [xxx@x.ru, yyy@y.com]

В итоге должен формироваться датафрейм со столбцами: <почта> - <дата утечки> - <источник утечки> - <описание утечки>.

In [24]:
URL = 'https://digibody.avast.com/v1/web/leaks'
json = {'email': 'mvr1976@gmail.com'}
rez_py = requests.post(URL, json = json)
rez1 = rez_py.json()
leak_list = rez1['value']

user = []
leak_date = []
leak_source = []
leak_descrip = []

for leak_dict in leak_list:
    user.append(leak_dict['username'])
    leak_date.append(datetime.datetime.fromtimestamp(leak_dict['leak_date']/1000.0))
    leak_source.append(leak_dict['domain'])
    leak_descrip.append(leak_dict['leak_info']['description'])


df = pd.DataFrame({'Почта': user, 'Дата утечки': leak_date, 'источник утечки': leak_source, 'описание утечки':leak_descrip})
df

,Почта,Дата утечки,источник утечки,описание утечки
0,mvr1976@gmail.com,2017-12-22 03:00:00,,The proliferation of stolen or leaked database...
1,old35lambert,2019-05-23 03:00:00,livejournal.com,"In 2017, social network LiveJournal's database..."
2,mvr1976@gmail.com,2019-02-06 03:00:00,,"On January 7, 2019, an online user named Sanix..."
3,mvr1976@gmail.com,2017-10-09 03:00:00,,The proliferation of stolen or leaked database...
4,mvr1976@gmail.com,2016-10-26 03:00:00,twitter.com,Login credentials for over 32 Million Twitter ...
5,mvr1976@gmail.com,2019-02-06 03:00:00,,"On January 7, 2019, an online user named Sanix..."
6,mvr1976@gmail.com,2019-02-06 03:00:00,,"On January 7, 2019, an online user named Sanix..."
7,mvr1976@gmail.com,2019-10-31 03:00:00,gatehub.net,"At an unconfirmed date, the online crypto curr..."
8,mvr1976@gmail.com,2017-11-04 03:00:00,myheritage.com,"In October 2017, a customer database belonging..."
9,mvr1976@gmail.com,2018-07-28 03:00:00,,This combolist was compiled from a variety of ...


In [46]:
EMAILs = ['mvr1976@gmail.com', 've-vo@mail.ru', 'goodyear_south@mail.ru']
URL = 'https://digibody.avast.com/v1/web/leaks'
mails_df = pd.DataFrame()
def check_leak(json):    
    user = []
    leak_date = []
    leak_source = []
    leak_descrip = []
    rez_py = requests.post(URL, json = json)
    rez1 = rez_py.json()
    leak_list = rez1['value']
    for leak_dict in leak_list:
        user.append(leak_dict['username'])
        leak_date.append(datetime.datetime.fromtimestamp(leak_dict['leak_date']/1000.0))
        leak_source.append(leak_dict['domain'])
        leak_descrip.append(leak_dict['leak_info']['description'])
    df = pd.DataFrame({'Почта': user, 'Дата утечки': leak_date, 'источник утечки': leak_source, 'описание утечки':leak_descrip})
    return df
    
for el in EMAILs:
    json = {'email': el}
    mails_df = pd.concat([mails_df, pd.DataFrame([check_leak(json)])])
mails_df

,0
0,Почта Дата утечки ист...
0,Почта Дата утечки источник...
0,"Empty DataFrame Columns: [Почта, Дата утечки, ..."


## Практика 2. Напишем скрипт, который будет собирать новости с сайта Коммерсанта

In [47]:
URL = 'https://www.kommersant.ru/search/results'
params = {
    'search_query': 'python'
}

In [48]:
res = requests.get(URL, params)

In [49]:
soup = BeautifulSoup(res.text, 'html.parser')
soup


<!DOCTYPE html>

<html class="no-js" lang="ru">
<head>
<title>Коммерсантъ: последние новости России и мира</title>
<meta content="IE=edge, chrome=1" http-equiv="X-UA-Compatible"/>
<meta charset="utf-8"/>
<script>document.write(document.cookie.indexOf('IsNonMobileViewport') >= 0 ? '<meta name="viewport" content="width=1200">' : '<meta name="viewport" content="width=device-width, initial-scale=1.0">')</script>
<meta content="telephone=no" name="format-detection"/>
<meta content="Коммерсантъ: последние новости России и мира" name="title"/>
<meta content="Актуальные новости, объективный анализ и эксклюзивные комментарии о важнейших событиях и трендах" name="description"/>
<meta content="Новости,Политика,Экономика,Бизнес,Финансы,Дело,Биржа,Рынок,Акции,Прогнозы,Критика,Интервью,Рейтинги,Документы,Деньги,Власть,Автопилот,Тематические страницы,Первые лица,Деловые новости,Мировая практика,Культура,Спорт,Weekend,Астрологический прогноз,Погода мира,Курсы валют ЦБ РФ" name="keywords"/>
<link href

In [50]:
# добираемся до блоков с новостями
news_blocks = soup.find_all('div', class_='search_results_item')
news_blocks

[<div class="search_results_item">
 <div class="text">
 <div class="search_results_stat">
 <span class="vicon vicon--article vicon--small">
 <svg class="vicon__body">
 <use xlink:href="#vicon-article" xmlns:xlink="http://www.w3.org/1999/xlink"></use>
 </svg>
 </span>
 <a href="/archive/online/163" target="_blank">
 <span class="search_results_number">001</span>
 <span class="search_results_source">Ъ-Волга-Урал-Online от 11.08.2020</span>
 </a>
 </div>
 <h2 class="article_name">
 <a href="/doc/4450177?query=python" target="_blank">Питерские и казанские программисты стали победителями хакатона DIGITALSUPERHERO</a>
 </h2>
 <div class="article_intro">
 <a href="/doc/4450177?query=python" target="_blank">
 									 ...  и методологии, применяя <mark>Python</mark>-разработку, технологии машинного ... 
 								</a>
 </div>
 </div>
 </div>,
 <div class="search_results_item search_results_item--media">
 <div class="search_results_media">
 <div class="photo">
 <a href="/doc/4434580?query=pytho

In [51]:
# добираемся до текста со ссылкой
articles_intro = list(map(lambda x: x.find('div', class_='article_intro'), news_blocks))
articles_intro

[<div class="article_intro">
 <a href="/doc/4450177?query=python" target="_blank">
 									 ...  и методологии, применяя <mark>Python</mark>-разработку, технологии машинного ... 
 								</a>
 </div>,
 <div class="article_intro">
 <a href="/doc/4434580?query=python" target="_blank">
 									 ... , как «Программирование на <mark>Python</mark>», «Мобильная разработка», «Разработка ... 
 								</a>
 </div>,
 <div class="article_intro">
 <a href="/doc/4407208?query=python" target="_blank">
 									 ...  и методологии, применяя <mark>Python</mark>-разработку, технологии машинного ... 
 								</a>
 </div>,
 <div class="article_intro">
 <a href="/doc/4399817?query=python" target="_blank">
 									 ... , как «Программирование на <mark>Python</mark>», «Мобильная разработка», «Разработка ... 
 								</a>
 </div>,
 <div class="article_intro">
 <a href="/doc/4377181?query=python" target="_blank">
 									 ...  сколько обойдется изучение <mark>Python</mark>, курсы по маркетингу ...

In [52]:
# добираемся до ссылок
a_list = list(map(lambda x: x.find('a').get('href'), articles_intro))
a_list

['/doc/4450177?query=python',
 '/doc/4434580?query=python',
 '/doc/4407208?query=python',
 '/doc/4399817?query=python',
 '/doc/4377181?query=python',
 '/doc/4349643?query=python',
 '/doc/4317759?query=python',
 '/doc/4282410?query=python',
 '/doc/4275981?query=python',
 '/doc/4155923?query=python']

In [79]:
# формируем полноценные ссылки
all_refs = list(map(lambda x: 'https://www.kommersant.ru/' + x, a_list))
all_refs

['https://www.kommersant.ru//doc/4450177?query=python',
 'https://www.kommersant.ru//doc/4434580?query=python',
 'https://www.kommersant.ru//doc/4407208?query=python',
 'https://www.kommersant.ru//doc/4399817?query=python',
 'https://www.kommersant.ru//doc/4377181?query=python',
 'https://www.kommersant.ru//doc/4349643?query=python',
 'https://www.kommersant.ru//doc/4317759?query=python',
 'https://www.kommersant.ru//doc/4282410?query=python',
 'https://www.kommersant.ru//doc/4275981?query=python',
 'https://www.kommersant.ru//doc/4155923?query=python']

In [90]:
# объединим все в одну функцию
def get_all_links(url, query):
    all_refs = []
    params = {
        'search_query': query,
    }
    res = requests.get(URL, params)
    time.sleep(0.3)
    soup = BeautifulSoup(res.text, 'html.parser')
    news_blocks = soup.find_all('div', class_='search_results_item')
    articles_intro = list(map(lambda x: x.find('div', class_='article_intro'), news_blocks))
    a_list = list(map(lambda x: x.find('a').get('href'), articles_intro))
    all_refs = list(map(lambda x: 'https://www.kommersant.ru/' + x, a_list))

    return all_refs

all_links = get_all_links(URL, 'python')
all_links

[]

In [ ]:
# но мы же собрали только одну страницу? Хотим ВСЕ новости
def get_all_links(url, query, pages):
    all_refs = []
    params = {
        'search_query': query
    }
    for i in range(pages):
        params['page'] = i + 1
        res = requests.get(URL, params)
        time.sleep(0.3)
        soup = BeautifulSoup(res.text, 'html.parser')
        news_blocks = soup.find_all('div', class_='search_results_item')
        articles_intro = list(map(lambda x: x.find('div', class_='article_intro'), news_blocks))
        a_list = list(map(lambda x: x.find('a').get('href'), articles_intro))
        all_refs += list(map(lambda x: 'https://www.kommersant.ru/' + x, a_list))
    return all_refs

all_links = get_all_links(URL, 'python', 3)
all_links

In [ ]:
import pandas as pd

In [ ]:
# собираем даты, заголовки и тексты новостей
# получаем ошибочку. Значит не у всех получаемых страниц одинаковая разметка
for link in all_links:
    soup = BeautifulSoup(requests.get(link).text, 'html.parser')
    time.sleep(0.3)
    date = pd.to_datetime(soup.find('time', class_='title__cake').get('datetime'), dayfirst=True).date()
    print(date)
    title = soup.find('h1', class_='article_name').text
    print(title)
    text = soup.find('div', class_='article_text_wrapper').text
    print(text)

In [ ]:
# получаем ошибочку. Значит не у всех получаемых страниц одинаковая разметка
for link in all_links:
    soup = BeautifulSoup(requests.get(link).text, 'html.parser')
    if soup.find('div', class_='b-article__publish_date'):
        date = pd.to_datetime(soup.find('div', class_='b-article__publish_date').find('time').get('datetime'), dayfirst=True).date()
    elif soup.find('time', class_='title__cake'):
        date = pd.to_datetime(soup.find('time', class_='title__cake').get('datetime'), dayfirst=True).date()
    print(date)
    if soup.find('h2', class_='article_name'): 
        title = soup.find('h2', class_='article_name').text
    else: 
        title = soup.find('h1', class_='article_name').text    
    print(title)
    text = soup.find('div', class_='article_text_wrapper').text
    print(text)

In [ ]:
# запишем данные в датафрейм
kom_news = pd.DataFrame()
for link in all_links:
    soup = BeautifulSoup(requests.get(link).text, 'html.parser')
    time.sleep(0.3)
    if soup.find('div', class_='b-article__publish_date'):
        date = pd.to_datetime(soup.find('div', class_='b-article__publish_date').find('time').get('datetime'), dayfirst=True).date()
    elif soup.find('time', class_='title__cake'):
        date = pd.to_datetime(soup.find('time', class_='title__cake').get('datetime'), dayfirst=True).date()
    if soup.find('h2', class_='article_name'): 
        title = soup.find('h2', class_='article_name').text
    else: 
        title = soup.find('h1', class_='article_name').text    
    text = soup.find('div', class_='article_text_wrapper').text
    row = {'date': date, 'title': title, 'text': text}
    kom_news = pd.concat([kom_news, pd.DataFrame([row])])  
kom_news

In [ ]:
# обернем в функцию 
def get_kom_news(links):
    kom_news = pd.DataFrame()
    for link in all_links:
        soup = BeautifulSoup(requests.get(link).text, 'html.parser')
        if soup.find('div', class_='b-article__publish_date'):
            date = pd.to_datetime(soup.find('div', class_='b-article__publish_date').find('time').get('datetime'), dayfirst=True).date()
        elif soup.find('time', class_='title__cake'):
            date = pd.to_datetime(soup.find('time', class_='title__cake').get('datetime'), dayfirst=True).date()
        if soup.find('h2', class_='article_name'): 
            title = soup.find('h2', class_='article_name').text
        else: 
            title = soup.find('h1', class_='article_name').text    
        text = soup.find('div', class_='article_text_wrapper').text
        row = {'date': date, 'title': title, 'text': text}
        kom_news = pd.concat([kom_news, pd.DataFrame([row])])  
    return kom_news

In [ ]:
kom_news = get_kom_news(all_links)
kom_news

# API

## Практика 3. Получим данные о песнях исполнителя при помощи  [API ITunes](https://developer.apple.com/library/archive/documentation/AudioVideo/Conceptual/iTuneSearchAPI/index.html)

In [ ]:
# https://developer.apple.com/library/archive/documentation/AudioVideo/Conceptual/iTuneSearchAPI/UnderstandingSearchResults.html#//apple_ref/doc/uid/TP40017632-CH8-SW1
URL = 'https://itunes.apple.com/search?term=jack+johnson'

In [ ]:
res = requests.get(URL)

In [ ]:
res.text

In [ ]:
res.json()

In [ ]:
params = {
    'term': 'шнуров',
    'limit': 200,
#     'offset': 2
}

In [ ]:
# pd.set_option('display.max_columns', 100)
res = requests.get(URL, params)
# res.json()

pd.DataFrame(res.json()['results'])

In [ ]:
params = {
    'term': 'лазарев',
    'limit': 60,
    'attribute': 'allArtistTerm',
    'country': 'ru'
}

In [ ]:
res = requests.get(URL, params)
res.json()

pd.DataFrame(res.json()['results'])

## Практика 4. Соберем сообщения из новостной ленты ВК по нужному запросу

In [ ]:
# https://vk.com/dev/manuals
# https://vk.com/dev/newsfeed.search
NEWSFEED_REQUEST = 'https://api.vk.com/method/newsfeed.search?'
TOKEN = '9df7991c9df7991c9df7991c329d86910d99df79df7991cc363a27748dcf7ad91284ef6'
VERSION = '5.103'
SLEEP = 0.33

In [ ]:
# обращаем внимание, что максимальное количество постов, 
# которые можно вытащить за раз, ограничено
params = {
    'access_token': TOKEN,
    'v': VERSION,
    'q': 'короновирус',
    'count': 200
}

In [ ]:
res = requests.get(NEWSFEED_REQUEST, params)
res

In [ ]:
res.text

In [ ]:
res.json()

In [ ]:
res.json()['response']['items']

In [ ]:
pd.DataFrame(res.json()['response']['items'])

In [ ]:
# соберем все доступные сообщения по запросу
newsfeed_df = pd.DataFrame()
while True:
    result = requests.get(NEWSFEED_REQUEST, params)
    time.sleep(0.33)
    newsfeed_df = pd.concat([newsfeed_df, pd.DataFrame(result.json()['response']['items'])])
    if 'next_from' in result.json()['response'].keys():
        params['start_from'] = result.json()['response']['next_from']
    else:
        break
newsfeed_df